In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.conf import SparkConf

In [2]:
conf = SparkConf()
conf.set('spark.jars', 'postgresql-42.7.3.jar')

In [3]:
spark = SparkSession.builder \
       .master('local[1]') \
       .appName('spark postgres') \
       .config(conf=conf) \
       .getOrCreate()

24/05/22 08:41:48 WARN Utils: Your hostname, DESKTOP-R4B4RHT resolves to a loopback address: 127.0.1.1; using 172.30.252.130 instead (on interface eth0)
24/05/22 08:41:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/22 08:41:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = pd.read_csv('pizza_sales.csv')

24/05/22 08:42:04 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
df.head(n=5)

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,01-01-2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,01-01-2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,01-01-2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,01-01-2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,01-01-2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [6]:
spark_df = spark.createDataFrame(df)

In [7]:
spark_df.show()

24/05/22 08:42:28 WARN TaskSetManager: Stage 0 contains a task of very large size (2682 KiB). The maximum recommended task size is 1000 KiB.


+--------+--------+--------------+--------+----------+----------+----------+-----------+----------+--------------+--------------------+--------------------+
|pizza_id|order_id| pizza_name_id|quantity|order_date|order_time|unit_price|total_price|pizza_size|pizza_category|   pizza_ingredients|          pizza_name|
+--------+--------+--------------+--------+----------+----------+----------+-----------+----------+--------------+--------------------+--------------------+
|       1|       1|    hawaiian_m|       1|01-01-2015|  11:38:36|     13.25|      13.25|         M|       Classic|Sliced Ham, Pinea...|  The Hawaiian Pizza|
|       2|       2| classic_dlx_m|       1|01-01-2015|  11:57:40|      16.0|       16.0|         M|       Classic|Pepperoni, Mushro...|The Classic Delux...|
|       3|       2| five_cheese_l|       1|01-01-2015|  11:57:40|      18.5|       18.5|         L|        Veggie|Mozzarella Cheese...|The Five Cheese P...|
|       4|       2|   ital_supr_l|       1|01-01-2015|  11

In [8]:
spark_df.schema

StructType([StructField('pizza_id', LongType(), True), StructField('order_id', LongType(), True), StructField('pizza_name_id', StringType(), True), StructField('quantity', LongType(), True), StructField('order_date', StringType(), True), StructField('order_time', StringType(), True), StructField('unit_price', DoubleType(), True), StructField('total_price', DoubleType(), True), StructField('pizza_size', StringType(), True), StructField('pizza_category', StringType(), True), StructField('pizza_ingredients', StringType(), True), StructField('pizza_name', StringType(), True)])

In [9]:
schema = types.StructType([
    types.StructField('pizza_id', types.IntegerType(), True), 
    types.StructField('order_id', types.IntegerType(), True), 
    types.StructField('pizza_name_id', types.StringType(), True), 
    types.StructField('quantity', types.IntegerType(), True), 
    types.StructField('order_date', types.TimestampType(), True), 
    types.StructField('order_time', types.TimestampType(), True), 
    types.StructField('unit_price', types.DoubleType(), True), 
    types.StructField('total_price', types.DoubleType(), True), 
    types.StructField('pizza_size', types.StringType(), True), 
    types.StructField('pizza_category', types.StringType(), True), 
    types.StructField('pizza_ingredients', types.StringType(), True), 
    types.StructField('pizza_name', types.StringType(), True)
    ])


In [10]:
spark_df = spark.read.option("header", "true").schema(schema).csv("pizza_sales.csv")

In [11]:
spark_df.printSchema()

root
 |-- pizza_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- pizza_name_id: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- order_time: timestamp (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- total_price: double (nullable = true)
 |-- pizza_size: string (nullable = true)
 |-- pizza_category: string (nullable = true)
 |-- pizza_ingredients: string (nullable = true)
 |-- pizza_name: string (nullable = true)



In [12]:
spark_df.write.format("jdbc") \
    .options(url="jdbc:postgresql://172.18.0.2:5432/pizzasales",
             dbtable="public.pizza_sales",
             user="root",
             password="root",
             driver="org.postgresql.Driver") \
    .mode('overwrite') \
    .save()
